In [1]:

from PIL import Image
from pi_heif import register_heif_opener
import numpy as np
import cv2
import imutils
import os
register_heif_opener()



In [2]:
im = Image.open('../receipts/IMG_6115.HEIC') # do whatever need with a Pillow image
img_orig = np.asarray(im)
# bgr to rgb
img_orig = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
img_orig = imutils.resize(img_orig, width=500)

## Image pre-processing


In [ ]:
from utils.bg_removers import extract_receipt_grabcut

files = os.listdir('../receipts')
image_paths = [f'../receipts/{file}' for file in files]
model_path = "isnet_dis.onnx"
for image_path in image_paths:
    im = Image.open(image_path) # do whatever need with a Pillow image
    img_orig = np.asarray(im)
    # bgr to rgb
    img_orig = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
    img_orig = imutils.resize(img_orig, width=500)
    res = extract_receipt_grabcut(img_orig)
    # mask grey to bgr
    # mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
   
    cv2.imshow("result", cv2.cvtColor(res, cv2.COLOR_RGB2BGR))

    cv2.waitKey(0)
    cv2.destroyAllWindows()
  

In [ ]:
from utils.bg_removers import process_receipt_image
# Process single image
image_path = '../receipts/IMG_6120.HEIC'
extracted_receipt = process_receipt_image(image_path)

if extracted_receipt is not None:
    # Display or save the result
    cv2.imshow('Extracted Receipt', cv2.cvtColor(extracted_receipt, cv2.COLOR_RGB2BGR))
    cv2.waitKey(0)
    # Or save:
    # cv2.imwrite('extracted_receipt.jpg', cv2.cvtColor(extracted_receipt, cv2.COLOR_RGB2BGR))

## Extraction

In [2]:
from doctr.models import ocr_predictor,kie_predictor

model = ocr_predictor(det_arch='fast_small', reco_arch='crnn_mobilenet_v3_small', pretrained=True,assume_straight_pages=False, detect_orientation=True,
               )      
                      
model.det_predictor.model.postprocessor.bin_thresh = 0.3
model.det_predictor.model.postprocessor.box_thresh = 0.1

/Users/ghost/mambaforge/envs/rie/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ghost/mambaforge/envs/rie/lib/python3.11/site-packages/doctr/models/utils/pytorch.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.seria

In [11]:
from glob import glob 
receipts_folder = 'receipts'
image_paths = glob(f'{receipts_folder}/*.HEIC')
images = [ cv2.imencode('.jpg',  np.asarray(Image.open(image_path) ) )[1].tobytes() for image_path in image_paths]

In [12]:
len(images)

15

In [ ]:
# save images as jpg
for i in range(len(images)):
    with open(f'../receipts/{files[i].split(".")[0]}.jpg', 'wb') as f:
        f.write(images[i])

In [13]:
from doctr.io import DocumentFile
inference_images = DocumentFile.from_images(images)

In [14]:
inference_result = model(inference_images)

In [15]:
inference_result.pages[0].export()

{'page_idx': 0,
 'dimensions': (4032, 3024),
 'orientation': {'value': -3, 'confidence': None},
 'language': {'value': None, 'confidence': None},
 'blocks': [{'geometry': array([[0.25170118, 0.11752722],
          [0.6425169 , 0.1000019 ],
          [0.6794483 , 0.9235718 ],
          [0.28863257, 0.94109714]], dtype=float32),
   'objectness_score': 0.9891125549013537,
   'lines': [{'geometry': array([[0.36816406, 0.11230469],
            [0.42285156, 0.11230469],
            [0.42285156, 0.12402344],
            [0.36816406, 0.12402344]], dtype=float32),
     'objectness_score': 0.9790723597896951,
     'words': [{'value': 'GHAM',
       'confidence': 0.6591412425041199,
       'geometry': ((0.36848958333333337, 0.1123046875),
        (0.4231770833333333, 0.1123046875),
        (0.4231770833333333, 0.1240234375),
        (0.36848958333333337, 0.1240234375)),
       'objectness_score': 0.9790723597896951,
       'crop_orientation': {'value': 0, 'confidence': 0.61}}]},
    {'geometry': 

In [ ]:
from utils.ocr_formatter import format_document
formatted_documents = [format_document(page.export()['blocks'][0]['lines']) for page in inference_result.pages]

In [58]:
formatted_documents

["      quOM    (\n     ashuslt\n\n         M&S\n\n           EST.1884\n       St Thomas Hospital Bridge Road\n      Westminster\n           London\n           SE1 7EH\n       Vmerksandspencer836 com\n\n      WWWREPRINT  ***\n    ***\n\n                     £3.30*\n 1 FLAT WHT Balance O£3.30 Pay\n Items: 1            £3.30\n\n Card ***PAYMENTOVED**X\n\n     ***CARDHOLDER RECEIPTXX*\nVisa Credit      TID: ****4165\nMID: ****2564    AUTH: 388190\nCARD: 485738XXXXXX3253\nREAD: CONTACTLESSPAN SEQ NO:01\nAID: A0000000031010\nNO CARDHOLDER VERIFICATION\nVISA Contactless\n\n               A a 7besqa TUO duo JTA\n                          of\n\n **     Join Sparks Today!\n           ****          :\n  Fancy the chance to get your shopping\n           for free? 9sio irinomiuont\n Want to know you're helping char ities? o3\n\n         Join Sparks at sdbesi nOVenera\n   marksandspencer e com/joinsparks smharwr\n      or download the M&S App\n\n Good things happen every time you shop\n\n   Please 

In [ ]:
from utils.gemini_receipt_processor import GeminiReceiptProcessor
from keys import gemini_api_key
processor = GeminiReceiptProcessor(model_name="gemini-1.5-flash",api_key=gemini_api_key)


In [ ]:
text_data = [processor.extract_data_from_text(formatted_document) for formatted_document in formatted_documents]

In [ ]:
from utils.data_processsor import get_dataframe_from_data
text_df = get_dataframe_from_data(text_data, image_paths)
text_df

In [ ]:
image_data = [processor.extract_data_from_image(image_path) for image_path in image_paths]

In [ ]:
from utils.data_processsor import get_dataframe_from_data
text_df = get_dataframe_from_data(image_data, image_paths)
text_df

In [ ]:
from utils.gpt_receipt_processor import GPTReceiptProcessor
from keys import openai_api_key

processor = GPTReceiptProcessor(api_key=openai_api_key, model_name="gpt-4o")

In [20]:
data = [processor.extract_data_from_text(formatted_document) for formatted_document in formatted_documents]


In [28]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
print(os.getenv('OPENAI_API_KEY'))


None


In [45]:
from utils.data_processsor import get_dataframe_from_data
df = get_dataframe_from_data(data,image_paths)

Error getting location from postcode: 'NoneType' object has no attribute 'latitude'


In [4]:
import pandas as pd

In [21]:
df = pd.read_pickle('receipts.pkl')

In [22]:
# order by datetime
df = df.sort_values('datetime')
df

,datetime,businessName,items,prices,quantities,totalItems,totalPrice,paymentMethod,cardNumber,cardType,...,time,day_of_week,weekend,day_of_month,week_of_month,time_of_day,image_path,device,month,year
0,2024-10-04 14:55:00,Pret A Manger (Europe) Ltd,"[Toastie Classic Cheese, Very Berry Croissant]","[5.65, 2.5]","[1, 1]",2,8.15,card,485739xxxxxx3933,Visa,...,14:55,Friday,0,4,1,afternoon,receipts/IMG_6102.HEIC,Apple iPhone 12 mini,October,2024.0
1,2024-10-11 14:45:00,Elior UK PLC,[Vending Sales],[3.65],[1],1,3.65,card,,,...,14:45,Friday,0,11,2,afternoon,receipts/IMG_6115.HEIC,Apple iPhone 12 mini,October,2024.0
2,2024-10-11 14:45:00,Elior UK PLC,[TL],[3.65],[1],1,3.65,card,,,...,14:45,Friday,0,11,2,afternoon,,,NaN,NaN
3,2024-10-11 15:33:00,KACHA BAZAR WHITECHAPEL,[OKRA LOOSE],[2.89],[1],1,2.89,card,XXXX XXXX XXXX 3933,VISA CREDIT,...,15:33,Friday,0,11,2,afternoon,receipts/IMG_6120.HEIC,Apple iPhone 12 mini,October,2024.0
5,2024-10-14 11:46:00,Royal London Hospital,"[Global Flavours, City Bake, Fresh Fruit, NHS ...","[8.55, 3.1, 0.91, -2.92]","[1, 1, 1, 1]",3,9.64,cash,,,...,11:46,Monday,0,14,3,morning,receipts/IMG_6128.HEIC,Apple iPhone 12 mini,October,2024.0
17,2024-10-14 11:46:00,Royal London Hospital,"[Global Flavours, City Bake, Fresh Fruit, NHS ...","[8.55, 3.1, 0.91, -2.92]","[1, 1, 1, 1]",3,9.64,cash,,,...,11:46,Monday,0,14,3,morning,,,NaN,NaN
4,2024-10-14 11:46:00,Royal London Hospital,"[Global Flavours, City Bake, Fresh Fruit, NHS ...","[8.55, 3.1, 0.91, -2.92]","[1, 1, 1, 1]",3,9.64,cash,,,...,11:46,Monday,0,14,3,morning,,,NaN,NaN
6,2024-10-15 16:08:00,Pure,[Get Vit 250ml],[3.7],[1],1,3.70,card,,,...,16:08,Tuesday,0,15,3,afternoon,receipts/IMG_6133.HEIC,Apple iPhone 12 mini,October,2024.0
7,2024-10-16 11:02:00,Costcutter NHS Trust Queens Hospital,"[ROASTED SALTED CASHEW, KINDER BUENO WHITE, IN...","[1.25, 2.29, 2.99]","[1, 1, 1]",3,6.53,card,,,...,11:02,Wednesday,0,16,3,morning,receipts/IMG_6203.HEIC,Apple iPhone 12 mini,October,2024.0
8,2024-10-21 12:12:00,Elior UK PLC,[BaguettesaSalads],[14.23],[1],1,14.23,card,,,...,12:12,Monday,0,21,4,afternoon,receipts/IMG_6211.HEIC,Apple iPhone 12 mini,October,2024.0


In [ ]:

# Order the columns 
df = df[['datetime','businessName','items','prices','quantities','totalItems','totalPrice','paymentMethod','cardNumber','cardType',
         'businessAddress', 'businessPostalCode', 'latitude', 'longitude', 'date', 'time', 'day_of_week', 'weekend', 'day_of_month',
       'week_of_month', 'time_of_day', 'image_path','device'
         ]]

df.to_pickle('receipts.pkl')

In [16]:
# columns
# Index(['date', 'time', 'items', 'prices', 'paymentMethod', 'cardNumber',
#        'cardType', 'quantities', 'totalItems', 'totalPrice', 'businessName',
#        'businessAddress', 'businessPostalCode', 'latitude', 'longitude',
#        'image_path', 'datetime', 'day_of_week', 'weekend', 'day_of_month',
#        'week_of_month', 'time_of_day', 'device'],
#       dtype='object')
average_spend_per_transaction = df['totalPrice'].mean()
# average spend per transaction for time of day 
average_spend_per_transaction_time_of_day = df.groupby('time_of_day')['totalPrice'].mean()
# average spend per transaction for day of week
average_spend_per_transaction_day_of_week = df.groupby('day_of_week')['totalPrice'].mean()
# average spend per transaction for weekday and weekend
average_spend_per_transaction_weekend = df.groupby('weekend')['totalPrice'].mean()
# average spend transaction per week of month 
average_spend_per_transaction_week_of_month = df.groupby('week_of_month')['totalPrice'].mean()

# average count of transactions 
average_count_per_transaction = df['totalPrice'].count()
# average count of transactions for time of day
average_count_per_transaction_time_of_day = df.groupby('time_of_day')['totalPrice'].count()
# average count of transactions for day of week
average_count_per_transaction_day_of_week = df.groupby('day_of_week')['totalPrice'].count()
# average count of transactions for weekday and weekend
average_count_per_transaction_weekend = df.groupby('weekend')['totalPrice'].count()
# average count of transactions per week of month
average_count_per_transaction_week_of_month = df.groupby('week_of_month')['totalPrice'].count()


# are we buying more higher priced items or more lower priced items
average_cost_per_item = df['totalPrice'].sum() / df['totalItems'].sum()

# add the count of transactions per postal code and the lat and long
average_spend_per_postal_code = df.groupby('businessPostalCode').agg({'totalPrice':'mean', 'businessPostalCode':'count', 'latitude':'mean', 'longitude':'mean'})
# change businessPostalCode to transactionCount totalPrice to averageTransactionAmount
average_spend_per_postal_code = average_spend_per_postal_code.rename(columns={'businessPostalCode':'transactionCount', 'totalPrice':'averageTransactionAmount'})

# data to create a bar plot of count of unqiue businesses
business_counts = df['businessName'].value_counts()
postal_code_counts = df['businessPostalCode'].value_counts()



/var/folders/1c/jwsk4s1j0nlfqzycgqdw_3fr0000gn/T/ipykernel_50114/325032300.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_spend_per_transaction_time_of_day = df.groupby('time_of_day')['totalPrice'].mean()
/var/folders/1c/jwsk4s1j0nlfqzycgqdw_3fr0000gn/T/ipykernel_50114/325032300.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_count_per_transaction_time_of_day = df.groupby('time_of_day')['totalPrice'].count()


In [18]:
average_count_per_transaction_time_of_day

time_of_day
morning      7
afternoon    8
evening      0
Name: totalPrice, dtype: int64

In [ ]:
# add the count of transactions per postal code and the lat and long
average_spend_per_postal_code = df.groupby('businessPostalCode').agg({'totalPrice':'mean', 'businessPostalCode':'count', 'latitude':'mean', 'longitude':'mean'})
# change businessPostalCode to transactionCount totalPrice to averageTransactionAmount
average_spend_per_postal_code = average_spend_per_postal_code.rename(columns={'businessPostalCode':'transactionCount', 'totalPrice':'averageTransactionAmount'})
average_spend_per_postal_code


,averageTransactionAmount,transactionCount,latitude,longitude
businessPostalCode,,,,
,50.0000,1,NaN,NaN
E1 1BB,5.8425,4,51.517942,-0.059559
E1 1BY,2.8900,1,51.519496,-0.059280
E1 4GL,7.7000,1,51.520730,-0.050650
E13 8SL,14.2300,1,51.522509,0.036228
RM7 OAG,6.5300,1,NaN,NaN
SE1 2QN,8.1500,1,51.505503,-0.084953
SE1 2TF,3.7000,1,51.504905,-0.084235
SE1 7EH,4.9000,2,51.498691,-0.119028


In [24]:
# add month (Jan,Feb,Mar)ie month names  and year columns
df['month'] = df['datetime'].dt.month_name()
df['year'] = df['datetime'].dt.year



In [13]:
df['month']

11     October
8      October
7      October
14     October
12     October
4      October
10     October
1      October
13     October
9     November
0     November
3     November
6     November
5     November
2     November
Name: month, dtype: object

In [28]:
# remove duplicates where datetime is same
df = df.drop_duplicates(subset='datetime')

In [26]:
month = 'October'

# total spend per day for a month
df[df['month'] == month]["totalPrice"].count()



12

In [29]:
df.to_pickle('receipts.pkl')